In [1]:
import os
import re
import json
import gdal
import datetime
import itertools
import shapefile
import numpy as np
import pandas as pd
import geopandas as gpd
from scipy import stats
from datetime import datetime
from itertools import product
import matplotlib.pyplot as plt
#import functions_ground_motion as fgm


In [2]:
with open("../file_with_paths.json") as file_with_paths :
    FILE_PATHS = json.load(file_with_paths)

print("The base output directory is {}".format(FILE_PATHS["ground_motion_failure"]))

ground_motion_dir = FILE_PATHS["ground_motion_csv"]
ground_motion_file = ground_motion_dir + ""


precip_dir = FILE_PATHS["rain_intensity_caliv_valid"]
out_dir = FILE_PATHS["time_series_ml"]

# Here comes the rain again
rain_dir = FILE_PATHS["rain_dir"]
rain_file = rain_dir + "2014-01-01_to_2019-12-31_Intensity.csv"

topo_dir = FILE_PATHS["topo_dir"]
slopefile = topo_dir + "eu_dem_AoI_epsg32633_SLOPE.bil"

# road file
roaddir = FILE_PATHS["road_dir"]
roadfile = roaddir + "Road_line.shp" # this is in EPSG:32633

merged_ground_motion_distance_dir = FILE_PATHS["ground_motion_csv"]
merged_ground_motion_distance_file = merged_ground_motion_distance_dir + "merged_result.csv"



The base output directory is /exports/csce/datastore/geos/groups/LSDTopoData/FORESEE/Data/GroundMotion/Failure/


In [3]:
########## read pixel position data ##################

ground_motion_distance_pxl = pd.read_csv(merged_ground_motion_distance_file)
ground_motion_distance_pxl = np.array(ground_motion_distance_pxl)

# the first column only has indices - we don't need that.
ground_motion_distance_pxl = ground_motion_distance_pxl[:,1:]

ground_motion_distance_df = pd.DataFrame({'ground_motion': ground_motion_distance_pxl[:, 0], 'time': ground_motion_distance_pxl[:, 1], 'slope': ground_motion_distance_pxl[:,2], 'rows': ground_motion_distance_pxl[:,3],'cols': ground_motion_distance_pxl[:,4], 'distance_to_road': ground_motion_distance_pxl[:,5]})


ground_motion_distance_df['time'] =  pd.to_datetime(ground_motion_distance_df['time'])


In [9]:
result_with_rain = result_with_rain.sort_values('time', ascending=True)

ground_motion_distance_df.tail(10)

,ground_motion,time,slope,rows,cols,distance_to_road
1425360,3.6050727208644235,2019-02-16,0.0636248,549,1010,180
1425361,3.434799247907636,2019-03-02,0.0636248,549,1010,180
1425362,3.3669288690189934,2019-04-03,0.0636248,549,1010,180
1425363,[1.08975638 1.36434698 1.67835335 1.93980158 2...,2017-01-25,0.135286,528,1164,26
1425364,[1.46596005 1.71655933 1.95822025 2.11598346 2...,2017-02-08,0.135286,528,1164,26
1425365,[1.45776093 1.79776093 2.19171525 2.6471243 3...,2017-02-24,0.135286,528,1164,26
1425366,[1.46318436 1.7397707 2.04060288 2.27089774 2...,2017-02-26,0.135286,528,1164,26
1425367,[ 2.93212728 3.44875093 3.97368208 4.393616...,2017-03-28,0.135286,528,1164,26
1425368,[1.44500026 1.73500026 2.09900575 2.58095369 3...,2017-03-30,0.135286,528,1164,26
1425369,[2.45794477 2.90347816 3.38009916 3.68498947 3...,2017-04-15,0.135286,528,1164,26


In [4]:
####### need to add the rainfall data to the datafram as well. forgot to do this ###########
rain_data = pd.read_csv(rain_file)
rain_data = np.array(rain_data)
rain_data_df = pd.DataFrame({'rain_intensity_mm_sec':rain_data[:,1]})
date_list = pd.date_range(start="2014-01-01", end="2018-12-31")

rain_data_df['time'] = date_list

print(rain_data_df.head())



   rain_intensity_mm_sec       time
0           0.000000e+00 2014-01-01
1           2.258827e-07 2014-01-02
2           1.985849e-06 2014-01-03
3           8.754348e-07 2014-01-04
4           1.807747e-04 2014-01-05


In [5]:
# note that the ground motion data goes all the way to end of 2019 but the rainfall data is only until end of 2018
result_with_rain = pd.merge(ground_motion_distance_df, rain_data_df, how='left', on=['time'])
#print(result_with_rain.head())
result_with_rain = result_with_rain.set_index('time')
result_with_rain = result_with_rain.sort_values('time', ascending=True)
#result_with_rain.to_csv(ground_motion_dir+'merged_result_with_rain.csv')
print(result_with_rain.info(verbose=False))
# remove rows which contain missing values - these will only correspond to 2019 values, we dont have rain data for those
result_with_rain = result_with_rain.dropna()
# will bring any non numeric values to nan - easy fix- need to refine
print(result_with_rain.info(verbose=False))
#result_with_rain['ground_motion'] = result_with_rain.to_numeric(result_with_rain['ground_motion'])
result_with_rain['ground_motion'] = result_with_rain['ground_motion'].apply(pd.to_numeric, errors='coerce')
result_with_rain = result_with_rain.dropna()



<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1425370 entries, 2014-12-31 to 2019-04-03
Columns: 6 entries, ground_motion to rain_intensity_mm_sec
dtypes: float64(1), object(5)
memory usage: 76.1+ MB
None
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1411426 entries, 2014-12-31 to 2018-12-28
Columns: 6 entries, ground_motion to rain_intensity_mm_sec
dtypes: float64(1), object(5)
memory usage: 75.4+ MB
None
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1392812 entries, 2014-12-31 to 2018-12-28
Columns: 4 entries, ground_motion to rain_intensity_mm_sec
dtypes: float64(2), object(2)
memory usage: 53.1+ MB
None
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 994649 entries, 2014-12-31 to 2018-12-28
Columns: 4 entries, ground_motion to rain_intensity_mm_sec
dtypes: float64(2), object(2)
memory usage: 37.9+ MB
None


In [ ]:

# drop "rows" and "cols" dataframe columns, each column is uniquely identified by the slope and distance to road. 
result_with_rain = result_with_rain.drop(['rows', 'cols'], axis=1)
print(result_with_rain.info(verbose=False))

# delete negative ground motion values 
result_with_rain = result_with_rain[result_with_rain['ground_motion'] >= 0]
print(result_with_rain.info(verbose=False))